# Compare a text-based vs a vision-based browser

Warning: this notebook is experimental, it probably won't work out of the box!

In [ ]:
!pip install "smolagents[litellm]" -q

In [1]:
import datasets
from run_gaia import answer_single_question


eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]

/Users/aymeric/venv/gaia/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Make sure you deactivated Tailscale VPN, else some URLs will be blocked!


Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


Loaded evaluation dataset:
task
2    86
1    53
3    26
Name: count, dtype: int64


Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [2]:
to_keep = [
    "What's the last line of the rhyme under the flavor",
    'Of the authors (First M. Last) that worked on the paper "Pie Menus or Linear Menus',
    "In Series 9, Episode 11 of Doctor Who, the Doctor is trapped inside an ever-shifting maze. What is this location called in the official script for the episode? Give the setting exactly as it appears in the first scene heading.",
    "Which contributor to the version of OpenCV where support was added for the Mask-RCNN model has the same name as a former Chinese head of government when the names are transliterated to the Latin alphabet?",
    "The photograph in the Whitney Museum of American Art's collection with accession number 2022.128 shows a person holding a book. Which military unit did the author of this book join in 1813? Answer without using articles.",
    "I went to Virtue restaurant & bar in Chicago for my birthday on March 22, 2021 and the main course I had was delicious! Unfortunately, when I went back about a month later on April 21, it was no longer on the dinner menu.",
    "In Emily Midkiff's June 2014 article in a journal named for the one of Hreidmar's ",
    "Under DDC 633 on Bielefeld University Library's BASE, as of 2020",
    "In the 2018 VSCode blog post on replit.com, what was the command they clicked on in the last video to remove extra lines?",
    "The Metropolitan Museum of Art has a portrait in its collection with an accession number of 29.100.5. Of the consecrators and co-consecrators",
    "In Nature journal's Scientific Reports conference proceedings from 2012, in the article that did not mention plasmons or plasmonics, what nano-compound is studied?",
    'In the year 2022, and before December, what does "R" stand for in the three core policies of the type of content',
    "Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?",
]
eval_ds = eval_ds.filter(lambda row: any([el in row["Question"] for el in to_keep]))
eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})

In [3]:
import os

from dotenv import load_dotenv
from huggingface_hub import login


load_dotenv(override=True)

login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Text browser

In [ ]:
from scripts.text_inspector_tool import TextInspectorTool
from scripts.text_web_browser import (
    ArchiveSearchTool,
    FinderTool,
    FindNextTool,
    NavigationalSearchTool,
    PageDownTool,
    PageUpTool,
    SearchInformationTool,
    VisitTool,
)
from scripts.visual_qa import VisualQAGPT4Tool

from smolagents import CodeAgent, LiteLLMModel


proprietary_model = LiteLLMModel("gpt-4o")

In [ ]:
### BUILD AGENTS & TOOLS

WEB_TOOLS = [
    SearchInformationTool(),
    NavigationalSearchTool(),
    VisitTool(),
    PageUpTool(),
    PageDownTool(),
    FinderTool(),
    FindNextTool(),
    ArchiveSearchTool(),
]


surfer_agent = CodeAgent(
    model=proprietary_model,
    tools=WEB_TOOLS,
    max_steps=20,
    verbosity_level=2,
)

results_text = answer_questions(
    eval_ds,
    surfer_agent,
    "code_gpt4o_27-01_text",
    reformulation_model=proprietary_model,
    output_folder="output_browsers",
    visual_inspection_tool=VisualQAGPT4Tool(),
    text_inspector_tool=TextInspectorTool(proprietary_model, 40000),
)

### Vision browser

In [ ]:
!pip install helium -q

In [ ]:
from scripts.visual_qa import VisualQAGPT4Tool

from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel
from smolagents.vision_web_browser import (
    close_popups,
    go_back,
    helium_instructions,
    initialize_agent,
    save_screenshot,
    search_item_ctrl_f,
)


proprietary_model = LiteLLMModel("gpt-4o")
vision_browser_agent = initialize_agent(proprietary_model)
### BUILD AGENTS & TOOLS

CodeAgent(
    tools=[DuckDuckGoSearchTool(), go_back, close_popups, search_item_ctrl_f],
    model=proprietary_model,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
    max_steps=20,
    verbosity_level=2,
)

results_vision = answer_questions(
    eval_ds,
    vision_browser_agent,
    "code_gpt4o_27-01_vision",
    reformulation_model=proprietary_model,
    output_folder="output_browsers",
    visual_inspection_tool=VisualQAGPT4Tool(),
    text_inspector_tool=TextInspectorTool(proprietary_model, 40000),
    postprompt=helium_instructions
    + "Any web browser controls won't work on .pdf urls, rather use the tool 'inspect_file_as_text' to read them",
)

### Browser-use browser

In [ ]:
!pip install browser-use lxml_html_clean -q
!playwright install

In [ ]:
import asyncio

import nest_asyncio


nest_asyncio.apply()

from browser_use import Agent
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


class BrowserUseAgent:
    logs = []

    def write_inner_memory_from_logs(self, summary_mode):
        return self.results

    def run(self, task, **kwargs):
        agent = Agent(
            task=task,
            llm=ChatOpenAI(model="gpt-4o"),
        )
        self.results = asyncio.get_event_loop().run_until_complete(agent.run())
        return self.results.history[-1].result[0].extracted_content


browser_use_agent = BrowserUseAgent()

results_browseruse = answer_questions(
    eval_ds,
    browser_use_agent,
    "gpt-4o_27-01_browseruse",
    reformulation_model=proprietary_model,
    output_folder="output_browsers",
    visual_inspection_tool=VisualQAGPT4Tool(),
    text_inspector_tool=TextInspectorTool(proprietary_model, 40000),
    postprompt="",
    run_simple=True,
)

### Omniparser vision agent

In [2]:
!pip install ultralytics easyocr paddleocr paddlepaddle numpy==1.26.4 supervision==0.18.0 -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from scripts.vision_web_browser import make_vision_agent
from scripts.visual_qa import visualizer
import pandas as pd
from smolagents import LiteLLMModel

instructions = """
Use your web_search tool when you want to get Google search results.
Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!
Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"

You can navigate to pages.
Code:
```py
go_to('github.com/trending')
```<end_code>

You can directly click a box by giving its number
Code:
```py
click_box(18)
```<end_code>

In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.

To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>

When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>

You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>

Proceed in several steps rather than trying to solve the task in one shot.
And at the end, only when you have your answer, return your final answer.
Code:
```py
final_answer("YOUR_ANSWER_HERE")
```<end_code>

If pages seem stuck on loading, you might have to wait, for instance `import time` and run `time.sleep(5.0)`. But don't overuse this!
To list elements on page, DO NOT try code-based element searches like 'contributors = find_all(S("ol > li"))': just look at the latest screenshot you have and read it visually, or use your tool search_item_ctrl_f.
Of course, you can act on buttons like a user would do when navigating.
After each code blob you write, you will be automatically provided with an updated screenshot of the browser and the current browser url.
But beware that the screenshot will only be taken at the end of the whole action, it won't see intermediate states.
Don't kill the browser.
When you have modals or cookie banners on screen, you should get rid of them before you can click anything else.
Any web browser controls won't work on .pdf urls, rather use the tool 'inspect_file_as_text' to read them
"""

vision_browser_agent = make_vision_agent()

model = LiteLLMModel("gpt-4o")

results = []
for example in eval_ds:
    example["question"] += instructions
    answer = answer_single_question(example, model, "out.jsonl", visualizer)
    results += answer

results_omniparser = pd.DataFrame(results)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You have one question to answer. It is paramount that you provide a correct answer.                             │
│ Give it all you can: I know for a fact that you have access to all the relevant tools to solve it and find the  │
│ correct answer (the answer does exist). Failure or 'I cannot answer' or 'None found' will not be tolerated,     │
│ success will be rewarded.                                                                                       │
│ Run verification steps if that's needed, you must make sure you find the correct answer!                        │
│ Here is the task:                                                                                               │
│ What's the last line of the rhyme under the flavor name on the headstone visible in the background of the photo │
│ of the oldest flavor's headstone in the Ben & Jerry's online flavor graveyard as of the end of 2022?            │
│ Use your web_search tool when you want to get Google search results.                                            │
│ Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!   │
│ Don't bother about the helium driver, it's already managed.                                                     │
│ We've already ran "from helium import *"                                                                        │
│                                                                                                                 │
│ You can navigate to pages.                                                                                      │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ go_to('github.com/trending')                                                                                    │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ You can directly click a box by giving its number                                                               │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click_box(18)                                                                                                   │
│ ```<end_code>                                                                                                   │
│                                                                                                                 │
│ In general stop your action after each button click to see what happens on your screenshot.                     │
│ Never try to login in a page.                                                                                   │
│                                                                                                                 │
│ To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.     │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ scroll_down(num_pixels=1200) # This will scroll one viewport down                                               │
│ ```<end_code>                                         

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                ```
1. Use the `web_search` tool to find the URL for the Ben & Jerry's flavor graveyard page.

2. Use the `helium` tool to navigate to the Ben & Jerry's flavor graveyard page URL and close any pop-ups if 
present using `close_popups`.

3. Explore the Ben & Jerry's flavor graveyard page to identify the section or listing of their oldest flavors as of
the end of 2022. 

4. Verify from available data on the page or accompanying information which flavor is the oldest. This might 
involve checking the dates associated with each flavor.

5. Visually locate the image or photo that features the headstone for the oldest flavor. This may require scrolling
through the page and identifying the correct image.

6. Once found, download or view the image containing the headstone.

7. Use the `visualizer` tool with the downloaded image to answer the question: "What is the last line of the rhyme 
under the flavor name on the headstone visible in this image?"

8. Verify the obtained last line with any available description or details on the webpage, ensuring accuracy.

9. Provide the final answer using the `final_answer` tool with the correct last line of the rhyme.

10. Complete the task with confirmation of the correct answer success.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Code:                                                                                                              
```py                                                                                                              
# Step 1: Use the web_search tool to find the URL for the Ben & Jerry's flavor graveyard page.                     
search_term = "Ben & Jerry's flavor graveyard"                                                                     
print(search_term)                                                                                                 
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Use the web_search tool to find the URL for the Ben & Jerry's flavor graveyard page.                   
  search_term = "Ben & Jerry's flavor graveyard"                                                                   
  print(search_term)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 3: Use the web_search tool with a broader query to find the Ben & Jerry's Flavor Graveyard URL.           
  search_agent(task="Locate the Ben & Jerry's Flavor Graveyard or a webpage featuring discontinued flavors.")      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search_agent'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Locate the Ben & Jerry's Flavor Graveyard or a webpage featuring discontinued flavors.                          │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.You can navigate to .txt online files.                                       │
│     If a non-html page is in another format, especially .pdf or a Youtube video, use tool                       │
│ 'inspect_file_as_text' to inspect it.                                                                           │
│     Additionally, if after some searching you find out that you need more information to answer the question,   │
│ you can use `final_answer` with your request for clarification as argument to request for more information.     │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

[Step 1: Duration 2.57 seconds| Input tokens: 9,426 | Output tokens: 114]

[Step 5: Duration 17.11 seconds| Input tokens: 34,276 | Output tokens: 634]

KeyboardInterrupt: 

### Get results

In [ ]:
import pandas as pd
from scripts.gaia_scorer import question_scorer


results_vision, results_text, results_browseruse = (
    pd.DataFrame(results_vision),
    pd.DataFrame(results_text),
    pd.DataFrame(results_browseruse),
)

results_vision["is_correct"] = results_vision.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)
results_text["is_correct"] = results_text.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)
results_browseruse["is_correct"] = results_browseruse.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)

In [ ]:
results = pd.concat([results_vision, results_text, results_browseruse])
results.groupby("agent_name")["is_correct"].mean()

In [ ]:
correct_vision_results = results_vision.loc[results_vision["is_correct"]]
correct_vision_results

In [ ]:
false_text_results = results_text.loc[~results_text["is_correct"]]
false_text_results